cnn
- 局部感知，每次卷积核覆盖一小部分像素，传统的神经网络是整体感知
- 权重共享，CNN除全连接层外，卷积层的参数完全取决于滤波器的设置大小,相比传统的参数量小，计算量小，整个图片共享一组滤波器的参数
- 多卷积核，卷积层会有多个卷积核，生成不同的特征

In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

输入层：一般是二维向量

卷积层：提取更高层次的特征，输入信号e(x)与滤波器函数h(x)进行卷积

池化层：下采样，减小数据处理量同时保留有用信息。取每邻域四个像素中的最大值变为一个像素，模糊图像。卷积已经提取出特征，相邻区域的特征是类似，近乎不变，池化只是选出最能表征特征的像素，缩减了数据量，同时保留了特征。

RELU层：激活函数

全连通层：高级特征进行全连接

输出层：结果预测，softmax

dropout，正则化方法，解决过拟合，随机删神经元。

In [4]:
# Convolutional neural network (two convolutional layers)

#  Conv2d padding:填充
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.2)
        )
        self.fc = nn.Linear(7*7*32, num_classes) # 全连接层
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2369
Epoch [1/5], Step [200/600], Loss: 0.0960
Epoch [1/5], Step [300/600], Loss: 0.1634
Epoch [1/5], Step [400/600], Loss: 0.1529
Epoch [1/5], Step [500/600], Loss: 0.0911
Epoch [1/5], Step [600/600], Loss: 0.0986
Epoch [2/5], Step [100/600], Loss: 0.0674
Epoch [2/5], Step [200/600], Loss: 0.0655
Epoch [2/5], Step [300/600], Loss: 0.1671
Epoch [2/5], Step [400/600], Loss: 0.1219
Epoch [2/5], Step [500/600], Loss: 0.1233
Epoch [2/5], Step [600/600], Loss: 0.0306
Epoch [3/5], Step [100/600], Loss: 0.0490
Epoch [3/5], Step [200/600], Loss: 0.0070
Epoch [3/5], Step [300/600], Loss: 0.1165
Epoch [3/5], Step [400/600], Loss: 0.0278
Epoch [3/5], Step [500/600], Loss: 0.0634
Epoch [3/5], Step [600/600], Loss: 0.0416
Epoch [4/5], Step [100/600], Loss: 0.0278
Epoch [4/5], Step [200/600], Loss: 0.0479
Epoch [4/5], Step [300/600], Loss: 0.0128
Epoch [4/5], Step [400/600], Loss: 0.0625
Epoch [4/5], Step [500/600], Loss: 0.0308
Epoch [4/5], Step [600/600], Loss:

In [6]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
# torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 99.01 %
